<h1><center>Tabular Playground Series -Jun 2021 </center></h1>

                                                      


<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:blue; border:0; color:white' role="tab" aria-controls="home"><center>Content</center></h2>

1. [Libraries](#libraries)  
2. [Load Datasets](#load_datasets)
3. [Data_Exploration](#data_exploration)



<a id="libraries"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:blue; border:0; color:white' role="tab" aria-controls="home"><center>Libraries</center></h2>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandas_profiling import ProfileReport
import seaborn as sns
pd.set_option('display.max_rows', None) #shows maximum rows
pd.set_option('display.max_columns', None) #shows maximum cloumns
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from xgboost import plot_importance
from matplotlib import pyplot
from sklearn.model_selection import train_test_split, cross_val_predict,cross_validate
from sklearn.linear_model import LogisticRegression #Load classifiers
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss
import optuna.integration.lightgbm as lgb
import optuna
from scipy import stats
from lightgbm import LGBMClassifier

<a id="load_datasets"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:blue; border:0; color:white' role="tab" aria-controls="home"><center>Load Datasets</center></h2>

In [ ]:
test =  pd.read_csv('../input/tabular-playground-series-jun-2021/test.csv')
train = pd.read_csv( '../input/tabular-playground-series-jun-2021/train.csv')
sample_submission = pd.read_csv('../input/tabular-playground-series-jun-2021/sample_submission.csv')

<a id="data_exploration"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:blue; border:0; color:white' role="tab" aria-controls="home"><center>Data exploration</center></h2>


In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
#copying the dataset into new ones for operation

df_train = train.copy()
df_test = test.copy()

In [ ]:
df_train.info()

In [ ]:
df_test.info()

### So, from the above information we can see that there are no missing values in test or train column

In [ ]:
#As Id column will has all unque values we will dropping it as of now.
df_train = df_train.drop(["id"], axis=1)
df_test = df_test.drop(["id"], axis=1)

In [ ]:
#Segregating numerical and ctargorical column for futhur exploration
feature_cols = df_train.columns

## Getting all the data that are not of "object" type. 
numerical_columns = df_train[feature_cols].select_dtypes(include=['int64','float64']).columns
categorical_columns = df_train[feature_cols].select_dtypes(exclude=['int64','float64','datetime64[ns]']).columns

print(len(numerical_columns), len(categorical_columns))

## Univariate Variable Analysis

### Categorical Variable

In [ ]:
def bar_plot(variable):
    """
        input: variable ex: "Gender"
        output: bar plot & value count
    """
    # get feature
    var = df_train[variable]
    # count number of categorical variable(value/sample)
    varValue = var.value_counts()
    
    # visualize
    plt.figure(figsize = (17,5))
    plt.bar(varValue.index, varValue)
    plt.xticks(varValue.index, varValue.index.values)
    plt.ylabel("Frequency")
    plt.title(variable)
    plt.show()
    print("{}:\n{}".format(variable,varValue))
    print("Number of Unique Values:",var.nunique())

In [ ]:
sns.set_style('darkgrid')

for c in categorical_columns:
    bar_plot(c)

### Replacing the catgorical values by number

In [ ]:


cleanup_nums = {"target": {"Class_1": 1, "Class_2": 2, "Class_3": 3, "Class_4": 4,
                                  "Class_5": 5, "Class_6": 6, "Class_7":7,"Class_8":8,"Class_9": 9 }}

df_train = df_train.replace(cleanup_nums)
df_train.head()

### Conclusion :

We will be building a classification task and the classification will be a multiclass imbalanced classification classification.


### Numerical variable

In [ ]:
def plot_hist(variable):
    plt.figure(figsize = (15,4)) 
    plt.subplot(1, 2, 1) # distribution
    plt.hist(df_train[variable], bins = 20)
    plt.xlabel(variable)
    plt.ylabel("Frequency")
    plt.title("{} distribution with hist".format(variable))

    plt.subplot(1, 2, 2) # outlier
    sns.boxplot(x=df_train[variable])
    plt.title("{} outlier distribution".format(variable))
    plt.show()
    
    print(df_train[variable].describe())

for n in numerical_columns:
    plot_hist(n)

### Conclusion

1. All of the columns has huge outliers but the amount is huge.So, we won't be removing as of now.
2. As there lot of column we might have to use PCA

### Correlations

In [ ]:
fig = plt.figure(figsize=(75, 75), dpi=100)
matrix = np.triu(df_train[numerical_columns].corr())
sns.heatmap(df_train[numerical_columns].corr(), annot=True, mask=matrix)
plt.title('New Features and Target Correlations', size=20, pad=20)

### Feature importance

In [ ]:
# split data into X and y
X = df_train.drop(['target'],axis=1)
y = df_train['target']

#making model
model = XGBClassifier()
model.fit(X, y)





In [ ]:
# get importance
importance = model.feature_importances_
# summarize feature importance

# plot feature importance
plt.figure(figsize=(20, 20))
plt.bar([x for x in range(len(importance))], importance)
plt.show()

### Conclusion:

We will take only the features which has importance more then 0.020

In [ ]:
for i,v in enumerate(importance):
   if v >= 0.02:
    print('Feature: %0d, Score: %.5f' % (i,v))

In [ ]:
df_train_updated = df_train[["feature_12", "feature_16","feature_18","feature_20","feature_21","feature_31","feature_37","feature_43","feature_56","target"]]
df_test_updated = df_test[["feature_12", "feature_16","feature_18","feature_20","feature_21","feature_31","feature_37","feature_43","feature_56"]]

In [ ]:
z_scores = stats.zscore(df_train_updated)
abs_z_scores = np.abs(z_scores)
filtered_entries = (abs_z_scores < 3).all(axis=1)
df_train_updated = df_train_updated[filtered_entries]


### Baseline model without outlier removal

In [ ]:
#Drop target variable from X and copy to y
X=df_train_updated.drop(['target'],axis=1)
y=df_train_updated['target']

### Hyper parameter tuning for LGBM using Optuna 

In [ ]:
def objective(trial, data = X, target = y):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 0)

    params = {
        'reg_alpha': trial.suggest_float('reg_alpha', 0.001, 10.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.001, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 11, 333),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'max_depth': trial.suggest_int('max_depth', 5, 30),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.01, 0.02, 0.05, 0.005, 0.1]),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 0.8),
        'n_estimators': trial.suggest_int('n_estimators', 50, 3000),
        'random_state': 42,
        'boosting_type': 'gbdt',
        'metric': 'multi_logloss',
        'num_class':9,
        'device': 'cpu',
        'objective': 'multiclass'
    }


    model = LGBMClassifier(**params)  
    model.fit(X_train, y_train, eval_set = [(X_val,y_val)], early_stopping_rounds = 222, verbose = True)
    y_pred = model.predict_proba(X_val)
    logloss = log_loss(y_val, y_pred)

    return logloss


In [ ]:
%%time
study = optuna.create_study(direction = 'maximize')
study.optimize(objective, n_trials = 5)
print('Number of finished trials:', len(study.trials))



In [ ]:
print('Best trial:', study.best_trial.params)
print('Best value:', study.best_value)

In [ ]:
#Train-test split (80:20)
X_train, X_test,y_train,y_test = train_test_split(X, y, test_size=0.2, random_state=2)

print("Train dataset shape: {0}, \nTest dataset shape: {1}".format(train_x.shape, test_x.shape))

In [ ]:
params = {
        'reg_alpha': 8.808455264971526, 
        'reg_lambda': 7.293577390227297, 
        'num_leaves': 301, 
        'min_child_samples': 58, 
        'max_depth': 10,
        'learning_rate': 0.1,
        'colsample_bytree': 0.21480718879686642, 
        'n_estimators': 2296
    }


model = LGBMClassifier(**params)  
model.fit(X_train, y_train, eval_set = [(X_test,y_test)], early_stopping_rounds = 20, verbose = True)
y_pred = model.predict_proba(X_test)
logloss = log_loss(y_test, y_pred)

print("log loss, ",logloss)

